# Inoculation Protocol

## Method

## Layout
![Inoculation Layout](Images/Inoculation_layout.PNG)

## Worklist Generation

We developed an internal-use package to automate worklist generation in a consistent and reliable way for all kinds of experimental designs. The package is called `worklistgen`. (This is a work in progress, as we continue adding new features and fixing bugs. If you have any suggestions, please let us know.)

### Usage Example

In [2]:
# import worklistgen and other necessary libraries
import pandas as pd
from worklistgen.core import Source, Destination, Worklist

import random
from itertools import combinations

We want to set up everything for creating our worklist.

First, let's define the species and media that we are going to use in our experiment.

In [3]:
# Define Species
species = ['EF', 'FP', 'BO', 'CD', 'DI', 'EL', 
           'BU', 'AC', 'GH', 'IJ', 'KL', 'MN', 
           'OP', 'QR', 'ST', 'UV', 'WX', 'YZ']
# Define Media
media = ['DM38', 'LB', 'M9', 'M9Glucose', 
         'M9Glycerol', 'M9Acetate', 'M9Propionate', 'M9Butyrate']

Now, we need to define our sources. We do this using the `Source` class. The `Source` class takes the following parameters:

- `name`: The name (prefix) of the source. This is used to identify the source in the worklist.
- `source_type`: The type of the source. This can be either `"species"` or `"media"`.
- `labware_type`: The type of labware used for the source. Different labware types have different predefined layouts.
- `input`: A list of inputs.

The main attribute of the `Source` class is `well_map`, which contains the information about the location of each input in the labware.

Note that we have two sources: one for the species and one for the media. The `species` source contains the species we are going to be using in our experiment, and the `media` source contains the media conditions we are going to be using in our experiment. The `species` source is a 96-well plate, and the `media` source is a 24-tube trough.

In [ ]:
# Define sources
source1 = Source(name='species_source', source_type='species', labware_type='96wellplate', input=species)
source2 = Source(name='media_source', source_type='media', labware_type='tuberack', input=media)

Lets check the `well_map` of our sources.

In [ ]:
source1.well_map.head(5)

,Source_type,Input,Well,Name,Labware_type
0,species,EF,A1,species_source-1,96wellplate
1,species,FP,B1,species_source-1,96wellplate
2,species,BO,C1,species_source-1,96wellplate
3,species,CD,D1,species_source-1,96wellplate
4,species,DI,E1,species_source-1,96wellplate


In [ ]:
source2.well_map.head(5)

,Source_type,Input,Well,Name,Labware_type
0,media,DM38,A1,media_source-1,tuberack
1,media,LB,A2,media_source-1,tuberack
2,media,M9,A3,media_source-1,tuberack
3,media,M9Glucose,A4,media_source-1,tuberack
4,media,M9Glycerol,A5,media_source-1,tuberack


Now, we are going to define the location of each of the experimental conditions in the labware. We do this using the `Destination` class. The `Destination` class takes the following parameters:
- `name`: The name (prefix) of the destination. This is used to identify the destination in the worklist.
- `labware_type`: The type of labware used for the destination. Different labware types have different predefined layouts.
- `experiements`: Data frame containing the experimental conditions. The first column should contain the species separated by "-" (`community`) and the second column should contain the media conditions separated by "-" (`condition`). The rest of the columns will be ignored.
- `blanks`: List of blanks to include.
- `replicates`: Number of replicates per experiemntal combination.
- `blanks_replicates`: Number of replicates per blank.

Once again, the most importat attribute of the `Destination` class is `well_map`, which contains the information about the location of each input in the labware.

Here we are going to generate the experiemnts data frame randomly, but you can also use your own table. The only requirement is that the first column contains the species separated by "-" (`community`) and the second column contains the media conditions separated by "-" (`condition`). The rest of the columns will be ignored.

In [ ]:
# Randomly generate communities
community_combinations = []
for size in range(1, len(species) + 1):
    community_combinations.extend(combinations(species, size))
random_communities = random.sample(community_combinations, 60)
communities = ['-'.join(community) for community in random_communities]

# Randomly generate conditions
condition_combinations = []
for size in range(1, len(media) + 1):
    condition_combinations.extend(combinations(media, size))
random_conditions = random.sample(condition_combinations, 60)
conditions = ['-'.join(condition) for condition in random_conditions]

# Build the experiments DataFrame
experiments = pd.DataFrame({
    'community': communities,
    'condition': conditions,
})
experiments.head(10)

,community,condition
0,FP-BO-BU-GH-QR-WX-YZ,DM38-LB-M9Acetate-M9Propionate
1,FP-CD-EL-GH-IJ-KL-MN-UV-YZ,DM38-LB-M9Glucose-M9Acetate-M9Propionate-M9But...
2,EF-IJ-KL-OP-ST,DM38-M9Glucose-M9Glycerol-M9Acetate-M9Propionate
3,EF-FP-BO-CD-EL-BU-IJ-OP-ST-UV-WX-YZ,DM38-M9-M9Glycerol-M9Propionate-M9Butyrate
4,EF-CD-KL-OP-YZ,M9Butyrate
5,FP-DI-EL-BU-GH-KL-MN-QR-UV-WX-YZ,M9-M9Acetate
6,EF-FP-BO-CD-EL-BU-QR-ST-WX-YZ,DM38-LB-M9-M9Glycerol-M9Acetate-M9Propionate-M...
7,FP-GH-IJ-MN-QR-ST-WX-YZ,LB-M9
8,FP-EL-BU-AC-GH-MN-ST-UV-WX,DM38-M9Glucose-M9Acetate
9,EF-FP-DI-BU-IJ-KL-OP-ST-YZ,LB-M9Glucose-M9Acetate-M9Propionate


Now, lets create or destination object and check its `well_map`.

In [9]:
# Define destination
destination = Destination(name='destination', labware_type='96wellplate', experiments=experiments, blanks=['M9Propionate', 'M9Butyrate'], replicates=3, blanks_replicates=2)
destination.well_map.head(10)

,Community,Condition,Replicate,Well,Name,Labware_type
0,blank,M9Propionate,1,A1,destination-1,96wellplate
1,blank,M9Propionate,2,B1,destination-1,96wellplate
2,blank,M9Butyrate,1,C1,destination-1,96wellplate
3,blank,M9Butyrate,2,D1,destination-1,96wellplate
4,FP-BO-BU-GH-QR-WX-YZ,DM38-LB-M9Acetate-M9Propionate,1,E1,destination-1,96wellplate
5,FP-BO-BU-GH-QR-WX-YZ,DM38-LB-M9Acetate-M9Propionate,2,F1,destination-1,96wellplate
6,FP-BO-BU-GH-QR-WX-YZ,DM38-LB-M9Acetate-M9Propionate,3,G1,destination-1,96wellplate
7,FP-CD-EL-GH-IJ-KL-MN-UV-YZ,DM38-LB-M9Glucose-M9Acetate-M9Propionate-M9But...,1,H1,destination-1,96wellplate
8,FP-CD-EL-GH-IJ-KL-MN-UV-YZ,DM38-LB-M9Glucose-M9Acetate-M9Propionate-M9But...,2,A2,destination-1,96wellplate
9,FP-CD-EL-GH-IJ-KL-MN-UV-YZ,DM38-LB-M9Glucose-M9Acetate-M9Propionate-M9But...,3,B2,destination-1,96wellplate


Finally, we create the `Worklist` object. The `Worklist` class takes the following parameters:
- `name`: The name of the worklist.
- `total_volume`: Total volume of each well.

Because we can have several sources and destinations, we need to add them to the worklist. We do this using the `add_source()` and `add_destination()` methods. After that, we can generate the worklist using the `generate_worklist()` method.

In [10]:
# Create worklist
worklist = Worklist(name='worklist', total_volume=100)
worklist.add_source(source1)
worklist.add_source(source2)
worklist.add_destination(destination)

# Generate worklist
worklist.generate_worklist()
worklist.worklist_df

,Source_plate_Name,Source_Plate_Type,Source_Plate_Well,Destination_plate_Name,Destination_Plate_Type,Destination_Plate_Well,Transfer_Volume
0,media_source-1,tuberack,A7,destination-1,96wellplate,A1,100.000000
1,media_source-1,tuberack,A7,destination-1,96wellplate,B1,100.000000
2,media_source-1,tuberack,A8,destination-1,96wellplate,C1,100.000000
3,media_source-1,tuberack,A8,destination-1,96wellplate,D1,100.000000
4,species_source-1,96wellplate,B1,destination-1,96wellplate,E1,14.285714
...,...,...,...,...,...,...,...
2324,species_source-1,96wellplate,F2,destination-2,96wellplate,H11,10.000000
2325,species_source-1,96wellplate,G2,destination-2,96wellplate,H11,10.000000
2326,media_source-1,tuberack,A2,destination-2,96wellplate,H11,33.333333
2327,media_source-1,tuberack,A7,destination-2,96wellplate,H11,33.333333
